### Film и crop в массив

на основе scikit-video

In [ ]:
!pip install scikit-video -q
!pip install einops -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00


# common_metrics_on_video_quality
https://github.com/JunyaoHu/common_metrics_on_video_quality/tree/main

In [ ]:
!pip install lpips -q
!git clone https://github.com/JunyaoHu/common_metrics_on_video_quality.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 76.6 MB/s eta 0:00:00


In [ ]:
import os
import torch
import sys
dir_project = '/content/common_metrics_on_video_quality'
sys.path.append(dir_project)

import skvideo.io
import numpy as np
np.float = np.float64
np.int = np.int_
import json
from calculate_fvd import calculate_fvd
from calculate_psnr import calculate_psnr
from calculate_ssim import calculate_ssim
from calculate_lpips import calculate_lpips
device = torch.device("cuda")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [on]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:03<00:00, 70.3MB/s]


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


In [ ]:
MAIN_DIR = " you main dir"
EVAL_DIR = MAIN_DIR + "/Increment_Dual"

MOVI_DIR = MAIN_DIR + "/MSR_VIT_movi"
ORIG_DIR = MOVI_DIR + "/crop_hand_selected_mp4"

SIZE = 256
QTYFRAMES = 16

def movi_numpy_to_tensor(array):
    return torch.tensor(array).permute(0, 3, 1, 2).unsqueeze(0)

## real_video

In [ ]:
video_list = os.listdir(ORIG_DIR)
video_list.sort()
print(len(video_list))

video_directory = ORIG_DIR +"/"

all_video = []
for film in video_list:
    vid = skvideo.io.vread(fname = video_directory + film,
                           num_frames = QTYFRAMES,
                           outputdict={
                              "-sws_flags": "bilinear",
                              "-s": f"{SIZE}x{SIZE}"
                          }
    )
    all_video.append(movi_numpy_to_tensor(vid))

real_video = torch.concat(all_video)
real_video.shape

31


torch.Size([31, 16, 3, 256, 256])

#  Fréchet Video Distance (FVD)

## predict_step_vectors_H512_W512_F16_S75_caption_uniform

In [ ]:
GEN_DIR = EVAL_DIR + "/predict_step_vectors_H512_W512_F16_S75_caption_uniform"
video_list = os.listdir(GEN_DIR)
video_list.sort()
print(len(video_list))

video_directory = GEN_DIR +"/"

all_video = []
for film in video_list:
    vid = skvideo.io.vread(fname = video_directory + film,
                           num_frames = QTYFRAMES,
                           outputdict={
                              "-sws_flags": "bilinear",
                              "-s": f"{SIZE}x{SIZE}"
                          }
    )
    all_video.append(movi_numpy_to_tensor(vid))

predict_step_video = torch.concat(all_video)
predict_step_video.shape

31


torch.Size([31, 16, 3, 256, 256])

In [ ]:
result = {}
result['fvd_VGPT'] = calculate_fvd(real_video/255, predict_step_video/255, device, method='videogpt')
result['fvd_GAN'] = calculate_fvd(real_video/255, predict_step_video/255, device, method='styleganv')
result['ssim'] = calculate_ssim(real_video/255, predict_step_video/255)
result['psnr'] = calculate_psnr(real_video/255, predict_step_video/255)
result['lpips'] = calculate_lpips(real_video/255, predict_step_video/255, device)
print(json.dumps(result, indent=4))

calculate_fvd...
/content/common_metrics_on_video_quality/fvd/videogpt/i3d_pretrained_400.pt


100%|██████████| 7/7 [00:33<00:00,  4.74s/it]


calculate_fvd...
/content/common_metrics_on_video_quality/fvd/styleganv/i3d_torchscript.pt


100%|██████████| 7/7 [00:31<00:00,  4.52s/it]


calculate_ssim...


...

calculate_psnr...


100%|██████████| 31/31 [00:00<00:00, 140.91it/s]


calculate_lpips...


100%|██████████| 31/31 [00:02<00:00, 14.23it/s]


{
    "fvd_VGPT": {
        "value": {
            "10": 2534.40625,
            "11": 2101.968505859375,
            "12": 2457.84765625,
            "13": 1955.55419921875,
            "14": 2112.8115234375,
            "15": 1761.67822265625,
            "16": 2056.290283203125
        },
        "video_setting": [
            31,
            3,
            16,
            256,
            256
        ],
        "video_setting_name": "batch_size, channel, time, heigth, width"
    },
    "fvd_GAN": {
        "value": {
            "10": -755914243657.979,
            "11": 54526285.678062655,
            "12": 2359708.1308414633,
            "13": 8796093022510.012,
            "14": 60817714.48576285,
            "15": 481036337409.45337,
            "16": -201326313.52976418
        },
        "video_setting": [
            31,
            3,
            16,
            256,
            256
        ],
        "video_setting_name": "batch_size, channel, time, heigth, width"
    },
 

In [ ]:
d = {       "10": 2534.40625,
            "11": 2101.968505859375,
            "12": 2457.84765625,
            "13": 1955.55419921875,
            "14": 2112.8115234375,
            "15": 1761.67822265625,
            "16": 2056.290283203125
                 }

res = np.array(list(d.values()))
print(f"FVD predict_step_video mean: {np.mean(res )}")
print(f"FVD predict_step_video std: {np.std(res )}")

FVD predict_step_video mean: 2140.079520089286
FVD predict_step_video std: 251.54491887633392


In [ ]:
d = {
            "0": 0.21341095114947914,
            "1": 0.21553388104722343,
            "2": 0.20933405542360703,
            "3": 0.21014859946386175,
            "4": 0.21172745753003352,
            "5": 0.2187414494915422,
            "6": 0.2161165263933569,
            "7": 0.2160265237138919,
            "8": 0.22005650510833205,
            "9": 0.228580322855022,
            "10": 0.22697439443402667,
            "11": 0.223898147268739,
            "12": 0.2233259752975541,
            "13": 0.22445250357375707,
            "14": 0.2265620786179131,
            "15": 0.22840532332582575
                 }

res = np.array(list(d.values()))
print(f"SSIM predict_step_video mean: {np.mean(res )}")
print(f"SSIM  predict_step_video std: {np.std(res )}")

SSIM predict_step_video mean: 0.21958091841838534
SSIM  predict_step_video std: 0.006394885982218388


In [ ]:
d = {
            "0": 8.468607173051385,
            "1": 8.473080918332258,
            "2": 8.305497774572302,
            "3": 8.04701918152022,
            "4": 8.139209734633273,
            "5": 8.353620992868619,
            "6": 8.301381289494232,
            "7": 8.341825850739944,
            "8": 8.456250979736762,
            "9": 8.617300666962999,
            "10": 8.581655863751953,
            "11": 8.35896628030152,
            "12": 8.34856624491386,
            "13": 8.338194577471722,
            "14": 8.332312859512276,
            "15": 8.123711414238336
                 }

res = np.array(list(d.values()))
print(f"PSNR predict_step_video mean: {np.mean(res )}")
print(f"PSNR  predict_step_video std: {np.std(res )}")

PSNR predict_step_video mean: 8.349200112631355
PSNR  predict_step_video std: 0.1495837846014455


In [ ]:
d = {
            "0": 0.7847349489888837,
            "1": 0.7857839984278525,
            "2": 0.785987538676108,
            "3": 0.7902473499698024,
            "4": 0.7980922729738297,
            "5": 0.8016526276065458,
            "6": 0.8086844528875043,
            "7": 0.8064121738556893,
            "8": 0.8092422120032772,
            "9": 0.8049064778512524,
            "10": 0.7983084340249339,
            "11": 0.8046685245729261,
            "12": 0.7989171147346497,
            "13": 0.7967674539935204,
            "14": 0.7936768551026622,
            "15": 0.7967947029298351
                 }

res = np.array(list(d.values()))
print(f"LPIPS predict_step_video mean: {np.mean(res )}")
print(f"LPIPS  predict_step_video std: {np.std(res )}")

LPIPS predict_step_video mean: 0.7978048211624545
LPIPS  predict_step_video std: 0.007779828410445639


## predict_regress_vectors_fix_text_emb_H512_W512_F16_S75_caption_uniform

In [ ]:
GEN_DIR = EVAL_DIR + "/predict_regress_vectors_fix_text_emb_H512_W512_F16_S75_caption_uniform"
video_list = os.listdir(GEN_DIR)
video_list.sort()
print(len(video_list))

video_directory = GEN_DIR +"/"

all_video = []
for film in video_list:
    vid = skvideo.io.vread(fname = video_directory + film,
                           num_frames = QTYFRAMES,
                           outputdict={
                              "-sws_flags": "bilinear",
                              "-s": f"{SIZE}x{SIZE}"
                          }
    )
    all_video.append(movi_numpy_to_tensor(vid))

predict_regress_vectors_fix_text_emb = torch.concat(all_video)
predict_regress_vectors_fix_text_emb.shape

31


torch.Size([31, 16, 3, 256, 256])

In [ ]:
result = {}
result['fvd_VGPT'] = calculate_fvd(real_video/255, predict_regress_vectors_fix_text_emb/255, device, method='videogpt')
result['fvd_GAN'] = calculate_fvd(real_video/255, predict_regress_vectors_fix_text_emb/255, device, method='styleganv')
result['ssim'] = calculate_ssim(real_video/255, predict_regress_vectors_fix_text_emb/255)
result['psnr'] = calculate_psnr(real_video/255, predict_regress_vectors_fix_text_emb/255)
result['lpips'] = calculate_lpips(real_video/255, predict_regress_vectors_fix_text_emb/255, device)
print(json.dumps(result, indent=4))

calculate_fvd...
/content/common_metrics_on_video_quality/fvd/videogpt/i3d_pretrained_400.pt


100%|██████████| 7/7 [00:33<00:00,  4.83s/it]


calculate_fvd...
/content/common_metrics_on_video_quality/fvd/styleganv/i3d_torchscript.pt


100%|██████████| 7/7 [00:27<00:00,  3.97s/it]


calculate_ssim...


...

calculate_psnr...


100%|██████████| 31/31 [00:00<00:00, 149.82it/s]


calculate_lpips...


100%|██████████| 31/31 [00:02<00:00, 14.63it/s]


{
    "fvd_VGPT": {
        "value": {
            "10": 2535.90576171875,
            "11": 2524.723876953125,
            "12": 2635.6298828125,
            "13": 2491.872314453125,
            "14": 2351.534423828125,
            "15": 2301.765380859375,
            "16": 2442.2314453125
        },
        "video_setting": [
            31,
            3,
            16,
            256,
            256
        ],
        "video_setting_name": "batch_size, channel, time, heigth, width"
    },
    "fvd_GAN": {
        "value": {
            "10": -8589934013.477366,
            "11": 3539460.1040621744,
            "12": -2620848.5668599447,
            "13": 27111.12919008259,
            "14": -158329674399282.72,
            "15": 51677046505920.305,
            "16": 893353198112.455
        },
        "video_setting": [
            31,
            3,
            16,
            256,
            256
        ],
        "video_setting_name": "batch_size, channel, time, heigth, widt

In [ ]:
d = {       "10": 2535.90576171875,
            "11": 2524.723876953125,
            "12": 2635.6298828125,
            "13": 2491.872314453125,
            "14": 2351.534423828125,
            "15": 2301.765380859375,
            "16": 2442.2314453125
                 }

res = np.array(list(d.values()))
print(f"FVD predict_step_video mean: {np.mean(res )}")
print(f"FVD predict_step_video std: {np.std(res )}")

FVD predict_step_video mean: 2469.0947265625
FVD predict_step_video std: 105.7978288739626


In [ ]:
d = {
            "0": 0.2099040090945439,
            "1": 0.19918740781198951,
            "2": 0.190245742020757,
            "3": 0.19516525996409637,
            "4": 0.19825936709082723,
            "5": 0.19686920980692596,
            "6": 0.20021833219137905,
            "7": 0.20059101071253554,
            "8": 0.2048987959362829,
            "9": 0.2100275477889107,
            "10": 0.2103556185611384,
            "11": 0.20560234369525981,
            "12": 0.199870855495062,
            "13": 0.19650237930788617,
            "14": 0.19654132906091817,
            "15": 0.19687489155902438
                 }

res = np.array(list(d.values()))
print(f"SSIM predict_step_video mean: {np.mean(res )}")
print(f"SSIM  predict_step_video std: {np.std(res )}")

SSIM predict_step_video mean: 0.20069463125609605
SSIM  predict_step_video std: 0.005700505003908446


In [ ]:
d = {
            "0": 8.453017496749025,
            "1": 8.23366057716114,
            "2": 7.969645135212106,
            "3": 7.835461509252559,
            "4": 7.914966009150576,
            "5": 8.05276549255682,
            "6": 8.05182681129053,
            "7": 7.967112496049377,
            "8": 7.989667503732083,
            "9": 8.041292472021096,
            "10": 7.930696670730757,
            "11": 7.831517416059434,
            "12": 7.625545485407749,
            "13": 7.469217042883236,
            "14": 7.349704867748825,
            "15": 7.121243222566537
                 }

res = np.array(list(d.values()))
print(f"PSNR predict_step_video mean: {np.mean(res )}")
print(f"PSNR  predict_step_video std: {np.std(res )}")

PSNR predict_step_video mean: 7.864833763035741
PSNR  predict_step_video std: 0.3222310480195545


In [ ]:
d = {
            "0": 0.7804507113272144,
            "1": 0.7937205376163605,
            "2": 0.7924744083035377,
            "3": 0.7929969391515178,
            "4": 0.7965720257451457,
            "5": 0.8016329984511098,
            "6": 0.8043049016306477,
            "7": 0.804297064581225,
            "8": 0.8097251038397512,
            "9": 0.8100204467773438,
            "10": 0.8045872046101478,
            "11": 0.8081293721352855,
            "12": 0.8082332822584337,
            "13": 0.8119454018531307,
            "14": 0.8130670209084788,
            "15": 0.8155550072270055
                 }

res = np.array(list(d.values()))
print(f"LPIPS predict_step_video mean: {np.mean(res )}")
print(f"LPIPS  predict_step_video std: {np.std(res )}")

LPIPS predict_step_video mean: 0.8029820266510209
LPIPS  predict_step_video std: 0.00915737276939999


# Frechet Inception Distance (FID)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q torchmetrics[image]
from torchmetrics.image.fid import FrechetInceptionDistance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 12.6 MB/s eta 0:00:00


## predict_step_vectors_H512_W512_F16_S75_caption_uniform

In [ ]:
fid_eval = []
for i in range(real_video.shape[0]):
    fid = FrechetInceptionDistance(normalize=True)
    real_images = real_video[i]
    fake_images = predict_step_video[i]
    fid.update(real_images, real=True)
    fid.update(fake_images, real=False)
    eval = float(fid.compute())
    fid_eval.append(eval)
    print(f"FID: {eval}")

fid_eval = np.array(fid_eval)
print(f"FID predict_step_video mean: {np.mean(fid_eval)}")
print(f"FID predict_step_video std: {np.std(fid_eval)}")

FID: 295.82373046875
FID: 383.5173645019531
FID: 427.59130859375
FID: 405.2149963378906
FID: 393.9374694824219
FID: 363.35687255859375
FID: 314.8146667480469
FID: 390.32025146484375
FID: 386.70208740234375
FID: 476.92913818359375
FID: 328.2794189453125
FID: 365.95660400390625
FID: 529.5738525390625
FID: 453.9518127441406
FID: 464.58538818359375
FID: 396.6828308105469
FID: 437.5496520996094
FID: 386.4870910644531
FID: 278.9209289550781
FID: 390.1234436035156
FID: 461.2196350097656
FID: 391.29248046875
FID: 493.1166687011719
FID: 408.9107971191406
FID: 307.3275451660156
FID: 542.3809204101562
FID: 367.92034912109375
FID: 426.8111267089844
FID: 568.6043701171875
FID: 449.8199768066406
FID: 474.1563415527344
FID predict_step_video mean: 411.6735199959047
FID predict_step_video std: 69.24464750078684


## predict_regress_vectors_fix_text_emb

In [ ]:
fid_eval = []
for i in range(real_video.shape[0]):
    fid = FrechetInceptionDistance(normalize=True)
    real_images = real_video[i]
    fake_images = predict_regress_vectors_fix_text_emb[i]
    fid.update(real_images, real=True)
    fid.update(fake_images, real=False)
    eval = float(fid.compute())
    fid_eval.append(eval)
    print(f"FID: {eval}")

fid_eval = np.array(fid_eval)
print(f"FID predict_step_video mean: {np.mean(fid_eval)}")
print(f"FID predict_step_video std: {np.std(fid_eval)}")

FID: 310.1498718261719
FID: 399.6379089355469
FID: 399.2915954589844
FID: 444.02362060546875
FID: 424.0483703613281
FID: 390.779052734375
FID: 336.6081237792969
FID: 420.357177734375
FID: 473.35308837890625
FID: 375.43377685546875
FID: 376.1000671386719
FID: 374.715087890625
FID: 463.5829772949219
FID: 484.6566162109375
FID: 564.7755737304688
FID: 363.0639343261719
FID: 488.76629638671875
FID: 349.227294921875
FID: 303.5423278808594
FID: 422.9721984863281
FID: 412.84521484375
FID: 385.3534851074219
FID: 467.9363098144531
FID: 488.1471252441406
FID: 310.3887023925781
FID: 597.2940673828125
FID: 403.9840393066406
FID: 429.6247253417969
FID: 524.4007568359375
FID: 434.36676025390625
FID: 518.1158447265625
FID predict_step_video mean: 423.7916771673387
FID predict_step_video std: 70.93629836134609
